In [13]:
from nba_api.stats.static import teams
import pandas as pd
import numpy as np
import sqlite3
import re
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
conn = sqlite3.connect("nba.db")
cursor = conn.cursor()

In [14]:
from nba_api.stats.endpoints import leaguegamefinder
szn_20_21 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2020-12-22",
                                                date_to_nullable="2021-7-22", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_21_22 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2021-10-19",
                                              date_to_nullable="2022-7-19",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_22_23 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2022-10-18",
                                              date_to_nullable="2023-7-12",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_23_24 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2023-10-24",
                                              date_to_nullable="2024-4-14",
                                              season_type_nullable="Regular Season",
                                              league_id_nullable="00")
szn_24_25 = leaguegamefinder.LeagueGameFinder(date_from_nullable="2024-10-22",
                                                date_to_nullable="2025-4-13", 
                                                season_type_nullable="Regular Season",
                                                league_id_nullable='00')
szn_20_21 = szn_20_21.get_data_frames()[0]
szn_20_21 = pd.DataFrame(szn_20_21)
gameid = szn_20_21['GAME_ID'].unique()
szn_21_22 = szn_21_22.get_data_frames()[0]
szn_21_22 = pd.DataFrame(szn_21_22)
szn_22_23 = szn_22_23.get_data_frames()[0]
szn_22_23 = pd.DataFrame(szn_22_23)
szn_23_24 = szn_23_24.get_data_frames()[0]
szn_23_24 = pd.DataFrame(szn_23_24)
szn_24_25 = szn_24_25.get_data_frames()[0]
szn_24_25 = pd.DataFrame(szn_24_25)

In [15]:
szn_20_21.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS'],
      dtype='object')

In [16]:
lst=[szn_20_21, szn_21_22, szn_22_23, szn_23_24, szn_24_25]
for x in lst:
    x["TotalFGMade"] = x.groupby('GAME_ID')['FGM'].transform('sum')
    x["TotalFGAtt"] = x.groupby('GAME_ID')['FGA'].transform('sum')
    x['Total3PM'] = x.groupby('GAME_ID')['FG3M'].transform('sum')
    x["TotalTOV"] = x.groupby('GAME_ID')['TOV'].transform('sum')
    x["TotalFTA"] = x.groupby('GAME_ID')['FTA'].transform('sum')
    x['TotalOREB'] = x.groupby('GAME_ID')['OREB'].transform('sum')
    x['TotalGamePoints'] = x.groupby('GAME_ID')['PTS'].transform('sum')
    x['OppOREB'] = x['TotalOREB'] - x['OREB']
    x['TDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
    x['TotalGameFTM'] = x.groupby('GAME_ID')['FTM'].transform('sum')
    x['TotalDREB'] = x.groupby('GAME_ID')['DREB'].transform('sum')
    x['TotalAST'] = x.groupby('GAME_ID')['AST'].transform('sum')
    x['TotalSTL'] = x.groupby('GAME_ID')['STL'].transform('sum')
    x['TotalBLK'] = x.groupby('GAME_ID')['BLK'].transform('sum')
    x['TotalPF'] = x.groupby('GAME_ID')['PF'].transform('sum')
    x['TotalTOV'] = x.groupby('GAME_ID')['TOV'].transform('sum')

In [17]:
def EFG(x):
    num = float(x['FGM']) + 0.5 * float(x['FG3M'])
    denom = float(x['FGA'])
    return round(num/denom, 3)
def TOVP(x):
    denom = float(x['FGA']) + 0.44 * x['FTA'] + float(x['TOV'])
    return round(float(x['TOV']) / denom, 4)
def OREBP(x):
    x['OPPDREB'] = x['TDREB'] - x['DREB']
    return float(x['OREB']) / (float(x['OREB'] + x['OPPDREB']))
def FTRATE(x):
    return float(x['FTA']) / float(x['FGA'])

In [18]:
def OppEFG(x):
    num = float(x["TotalFGMade"] - x["FGM"]) + 0.5 * float(x["Total3PM"] - x["FG3M"])
    denom = float(x['TotalFGAtt'] - x["FGA"])
    return round(num/denom, 3)
def OppTOVP(x):
    num = float(x['TotalTOV'] - x['TOV'])
    denom = float(x["TotalFGAtt"] - x["FGA"]) + 0.44 * float(x["TotalFTA"] - x["FTA"]) + num
    return round(num/denom, 3)
def DREBP(x):
    return float(x['DREB']) / float(x['DREB'] + x['OppOREB'])
def OppFTRATE(x):
    return float(x['TotalFTA'] - x['FTA']) / float(x['TotalFGAtt'] - x['FGA'])
def away(x):
    return x["MATCHUP"][:4]
def home(x):
    return x["MATCHUP"][-4:]
def teamPIE(x):
    teampie = (x['PTS'] + x['FGM'] + x['FTM'] - x['FGA'] - x['FTA'] + x['DREB'] + (0.5 * x['OREB']) + x['AST'] + x['STL'] + (x['BLK']*0.5) - x['PF'] - x['TOV']) 
    gamepie = (x['TotalGamePoints'] + x['TotalFGMade'] + x['TotalGameFTM'] - x['TotalFGAtt'] - x['TotalFTA'] + x['TotalDREB'] + (0.5 * x['TotalOREB']) + x['TotalAST'] + x['TotalSTL'] + (0.5 * x['TotalBLK']) - x['TotalPF'] - x['TotalTOV'])
    return round(float(teampie) / float(gamepie), 4)

In [19]:
for x in lst:
    x['EFG'] = x.apply(EFG, axis=1)
    x['TOVP'] = x.apply(TOVP, axis=1)
    x['OREBP'] = x.apply(OREBP, axis=1)
    x['FTRATE'] = x.apply(FTRATE, axis=1)
    x['OppEFG'] = x.apply(OppEFG, axis=1)
    x['OppTOVP'] = x.apply(OppTOVP, axis=1)
    x['DREBP'] = x.apply(DREBP, axis=1)
    x['OppFTRATE'] = x.apply(OppFTRATE, axis=1)
    x["AWAY"] = x.apply(away, axis=1)
    x["HOME"] = x.apply(home, axis=1)
    x['TEAMPIE'] = x.apply(teamPIE, axis=1)

In [20]:
szn_20_21.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_21_22.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'], inplace=True)
szn_22_23.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_23_24.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)
szn_24_25.drop(columns=['TDREB', 'OppOREB', 'TotalOREB', 'TotalFTA', 'TotalTOV','Total3PM', 'TotalFGAtt', 'TotalFGMade'],inplace=True)

In [21]:
szn_20_21.columns

Index(['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'PTS', 'FGM', 'FGA', 'FG_PCT',
       'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB',
       'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS',
       'TotalGamePoints', 'TotalGameFTM', 'TotalDREB', 'TotalAST', 'TotalSTL',
       'TotalBLK', 'TotalPF', 'EFG', 'TOVP', 'OREBP', 'FTRATE', 'OppEFG',
       'OppTOVP', 'DREBP', 'OppFTRATE', 'AWAY', 'HOME', 'TEAMPIE'],
      dtype='object')

In [ ]:

for i, x in enumerate(lst):
    x['_original_index'] = range(len(x))
    x_sorted = x.sort_values(['TEAM_ID', 'SEASON_ID', 'GAME_DATE']).reset_index(drop=True)
    ewma_cols = ['PTS', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT',
                 'OREB', 'DREB', 'AST', 'STL', 'TOV', 'PF', 'PLUS_MINUS', 'EFG', 'TOVP',
                 'OREBP', 'FTRATE', 'OppEFG', 'OppTOVP', 'DREBP', 'OppFTRATE', 'TEAMPIE']
    for col in ewma_cols:
        x_sorted[f"EWMA {col}"] = (
            x_sorted.groupby(['TEAM_ID', 'SEASON_ID'])[col]
            .apply(lambda x: x.shift(1).ewm(span=7, min_periods=7).mean())
            .reset_index(level=[0, 1], drop=True)
        )
    x_sorted = x_sorted.sort_values('_original_index').reset_index(drop=True)
    x_sorted = x_sorted.drop(columns=['_original_index'])
    lst[i] = x_sorted

In [23]:
szn_20_21 = lst[0]
szn_21_22 = lst[1]
szn_22_23 = lst[2]
szn_23_24 = lst[3]
szn_24_25 = lst[4]

In [24]:
def combine_game_rows(df):
    df = df.copy()
    df['is_home'] = df['MATCHUP'].str.contains('vs.', na=False)
    stat_columns = [col for col in df.columns if col not in 
                   ['SEASON_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 
                    'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'AWAY', 'HOME']]
    home_data = df[df['is_home'] == True].copy()
    away_data = df[df['is_home'] == False].copy()
    home_rename = {col: f'home_{col}' for col in stat_columns}
    home_rename.update({
        'TEAM_ID': 'home_TEAM_ID',
        'TEAM_ABBREVIATION': 'home_TEAM',
        'TEAM_NAME': 'home_TEAM_NAME',
        'WL': 'home_WL'
    })
    home_data = home_data.rename(columns=home_rename)
    away_rename = {col: f'away_{col}' for col in stat_columns}
    away_rename.update({
        'TEAM_ID': 'away_TEAM_ID',
        'TEAM_ABBREVIATION': 'away_TEAM',
        'TEAM_NAME': 'away_TEAM_NAME',
        'WL': 'away_WL'
    })
    away_data = away_data.rename(columns=away_rename)
    merge_cols = ['GAME_ID', 'GAME_DATE']
    if 'SEASON_ID' in df.columns:
        merge_cols.append('SEASON_ID')
    combined = pd.merge(
        home_data[merge_cols + [col for col in home_data.columns if col not in merge_cols]],
        away_data[merge_cols + [col for col in away_data.columns if col not in merge_cols]],
        on=merge_cols,
        how='inner'
    )
    combined['home_win'] = (combined['home_WL'] == 'W').astype(int)
    return combined
combined_seasons = []
for df in lst:
    combined_df = combine_game_rows(df)
    combined_seasons.append(combined_df)
szn_20_21_combined = combined_seasons[0]
szn_21_22_combined = combined_seasons[1]
szn_22_23_combined = combined_seasons[2]
szn_23_24_combined = combined_seasons[3]
szn_24_25_combined = combined_seasons[4]

In [26]:
szn_20_21_combined.to_sql("2020-2021 Team Data Combined", conn, if_exists="replace", index=False)
szn_21_22_combined.to_sql("2021-2022 Team Data Combined", conn, if_exists="replace", index=False)
szn_22_23_combined.to_sql("2022-2023 Team Data Combined", conn, if_exists="replace", index=False)
szn_23_24_combined.to_sql("2023-2024 Team Data Combined", conn, if_exists="replace", index=False)
szn_24_25_combined.to_sql("2024-2025 Team Data Combined", conn, if_exists="replace", index=False)

1225

In [29]:
lst = szn_24_25_combined.columns